In [1]:
library(lme4)
library(lmerTest)
library(ggplot2)
library(plotly)
library(parallel)
library(MuMIn)
library(mgcv)
library(car)
library(DHARMa)
library(mgcv)
library(dplyr)
library(tidyverse)
library(itsadug)

residual_test <- function(model){
hist(residuals(model), main = "Histogram of Residuals", xlab = "Residuals")
qqnorm(residuals(model))
qqline(residuals(model), col = "red")
# shapiro.test(residuals(model))
ks.test(residuals(model), "pnorm", mean = mean(residuals(model)), sd = sd(residuals(model)))}

get_r2 <- function(model, data){
  yhat <- predict(model, type = "response")
  r2_tjur <- mean(yhat[data$term_type  == "usual"]) - mean(yhat[data$term_type  == "bio"])
  print(r2_tjur)
}

Le chargement a n'ecessit'e le package : Matrix


Attachement du package : 'lmerTest'


L'objet suivant est masqu'e depuis 'package:lme4':

    lmer


L'objet suivant est masqu'e depuis 'package:stats':

    step



Attachement du package : 'plotly'


L'objet suivant est masqu'e depuis 'package:ggplot2':

    last_plot


L'objet suivant est masqu'e depuis 'package:stats':

    filter


L'objet suivant est masqu'e depuis 'package:graphics':

    layout


Le chargement a n'ecessit'e le package : nlme


Attachement du package : 'nlme'


L'objet suivant est masqu'e depuis 'package:lme4':

    lmList


This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Le chargement a n'ecessit'e le package : carData

This is DHARMa 0.4.7. For overview type '?DHARMa'. For recent changes, type news(package = 'DHARMa')


Attachement du package : 'dplyr'


L'objet suivant est masqu'e depuis 'package:car':

    recode


L'objet suivant est masqu'e depuis 'package:nlme':

    collapse


Les objets su

In [9]:
df <- read.csv("compare_surprisals.csv")

df$term_group <- as.factor(df$term_group)
df$term <- as.factor(df$term)
df$surprisal_semantic[is.infinite(df$surprisal_semantic)] <- NA
df$surprisal_llm[is.infinite(df$surprisal_llm)] <- NA
df$surprisal_pmcllama[is.infinite(df$surprisal_pmcllama)] <- NA
df$surprisal_ngram[is.infinite(df$surprisal_ngram)] <- NA

df$surprisal_semantic_z <- scale(df$surprisal_semantic, center = TRUE, scale = TRUE)
df$surprisal_llm_z <- scale(df$surprisal_llm, center = TRUE, scale = TRUE)
df$surprisal_pmcllama_z <- scale(df$surprisal_pmcllama, center = TRUE, scale = TRUE)
df$surprisal_ngram_z <- scale(df$surprisal_ngram, center = TRUE, scale = TRUE)
df$frequency <- df$frequency
df$frequency_z <- scale(df$frequency, center = TRUE, scale = TRUE)
df <- na.omit(df)

In [10]:
df <- read.csv("results.csv")
# df$frequency <- log(df$frequency + 1)
df$term_type <- as.factor(df$term_type)
contrasts(df$term_type) <- c(0.5, -0.5)
print(contrasts(df$term_type))
df$term_group <- as.factor(df$term_group)
df$term <- as.factor(df$term)
df$surprisal[is.infinite(df$surprisal)] <- NA
df <- na.omit(df)

      [,1]
bio    0.5
usual -0.5


In [18]:
df_semantic <- df[df$surprisal_type=="semantic",]
df_semantic$frequency_z <- scale(df_semantic$frequency)
df_semantic$surprisal_z <- scale(df_semantic$surprisal)

model_semantic <- glmer(term_type ~ surprisal_z*frequency_z + (1 | term_group), data = df_semantic, family=binomial, control = glmerControl(optimizer = "Nelder_Mead", optCtrl = list(maxfun = 1000000)))
summary(model_semantic)
r.squaredGLMM(model_semantic)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: term_type ~ surprisal_z * frequency_z + (1 | term_group)
   Data: df_semantic
Control: 
glmerControl(optimizer = "Nelder_Mead", optCtrl = list(maxfun = 1e+06))

     AIC      BIC   logLik deviance df.resid 
   654.5    690.9   -322.2    644.5    10724 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-101946       0       0       0      30 

Random effects:
 Groups     Name        Variance Std.Dev.
 term_group (Intercept) 82.25    9.069   
Number of obs: 10729, groups:  term_group, 25

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              0.54907    0.04654  11.799  < 2e-16 ***
surprisal_z              0.27283    0.03948   6.911 4.83e-12 ***
frequency_z              7.26936    0.05225 139.138  < 2e-16 ***
surprisal_z:frequency_z -3.41611    0.03524 -96.939  < 2e-16 ***
---
Signif. codes:  0 '**

Warning message:
"the null model is only correct if all the variables it uses are identical 
to those used in fitting the original model."


,R2m,R2c
theoretical,0.4310797,0.9781199
delta,0.4235984,0.9611449


In [5]:
df_llm <- df[df$surprisal_type=="llm",]
df_llm$frequency_z <- scale(df_llm$frequency)
df_llm$surprisal_z <- scale(df_llm$surprisal)

model_llm <- glmer(term_type ~ surprisal_z*frequency_z + (1 | term_group), data = df_llm, binomial)
summary(model_llm)

r.squaredGLMM(model_llm)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: term_type ~ surprisal_z * frequency_z + (1 | term_group)
   Data: df_llm

     AIC      BIC   logLik deviance df.resid 
  2100.3   2136.7  -1045.2   2090.3    10724 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
 -267.1    -0.1     0.0     0.0 16979.7 

Random effects:
 Groups     Name        Variance Std.Dev.
 term_group (Intercept) 11.59    3.404   
Number of obs: 10729, groups:  term_group, 25

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -3.76131    0.71145  -5.287 1.24e-07 ***
surprisal_z             -1.50752    0.11833 -12.740  < 2e-16 ***
frequency_z              0.07369    0.09311   0.791    0.429    
surprisal_z:frequency_z  1.32925    0.10089  13.175  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (I

Warning message:
"the null model is only correct if all the variables it uses are identical 
to those used in fitting the original model."


,R2m,R2c
theoretical,0.3205097,0.8497646
delta,0.2858459,0.7578609


In [6]:
df_ngram <- df[df$surprisal_type=="ngram",]
df_ngram$frequency_z <- scale(df_ngram$frequency)
df_ngram$surprisal_z <- scale(df_ngram$surprisal)

model_ngram <- glmer(term_type ~ surprisal_z*frequency_z + (1 | term_group), data = df_ngram, binomial)
summary(model_ngram)

r.squaredGLMM(model_ngram)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: term_type ~ surprisal_z * frequency_z + (1 | term_group)
   Data: df_ngram

     AIC      BIC   logLik deviance df.resid 
  5789.6   5826.0  -2889.8   5779.6    10724 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7986 -0.2490 -0.1962 -0.1036 17.2537 

Random effects:
 Groups     Name        Variance Std.Dev.
 term_group (Intercept) 3.706    1.925   
Number of obs: 10729, groups:  term_group, 25

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.15328    0.39457  -2.923  0.00347 ** 
surprisal_z              0.02402    0.03396   0.707  0.47941    
frequency_z              0.85342    0.04808  17.749  < 2e-16 ***
surprisal_z:frequency_z -0.02655    0.03132  -0.848  0.39654    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            

Warning message:
"the null model is only correct if all the variables it uses are identical 
to those used in fitting the original model."


,R2m,R2c
theoretical,0.09442353,0.5741664
delta,0.07026998,0.4272946


In [19]:
AIC(model_semantic, model_pmcllama, model_llm, model_ngram)

,df,AIC
,<dbl>,<dbl>
model_semantic,5,654.4707
model_pmcllama,5,2190.6855
model_llm,5,2100.3251
model_ngram,5,5789.6167
